# Învățare Automată
# Arbori de decizie. Păduri aleatoare
### Autori:
* Tudor Berariu - 2016
* George Muraru - 2020

## 1. Scopul laboratorului

Scopul laboratorului îl reprezintă întelegerea conceptului de arbore de decizie și implementarea unor clasificatori bazați pe acest model.

## 2. Problema de rezolvat

Problema de rezolvat ı̂n acest laborator este una de ı̂nvățare supervizată: fiind dat un **set de date X** ce conține exemple descrise printr-un set de **atribute discrete A** și etichetate cu **câte o clasă dintr-o mulțime cunoscută C**, să se construiască un model pentru clasificarea exemplelor noi.

## 3. Arbore de decizie


Un arbore de decizie este un clasificator ce aproximează funcții discrete.

Într-un arbore de decizie există 2 tipuri de noduri:
* *noduri intermediare* - conține un test pentru un atribut și are câte un arc (și implicit un subarbore) pentru fiecare valoare posibiliă a atributului
* *noduri frunză* - este etichetat cu o clasă

Pentru **a clasifica un obiect nou** se pornește din rădăcina arborelui și din fiecare nod se coboară pe arcul corespunzător valorii atributului pe care o are obiectul dat. Atunci când se ajunge ı̂ntr-un nod frunză, clasa acestuia va reprezenta predicția arborelui.

## 4. Păduri de arbori aleatori

*Pădurile de arbori aleatori* (eng. Random Forest) este un model format din mai mulți arbori de decizie.

Se bazează pe 2 hiperparametrii:
* Eșantionare aleatoare din setul de date de antrenament
* Subseturi aleatoare de atribute considerate la împărțirea pe mai multi subarbori

Predicția, utilizând un astfel de model, se bazează pe clasa majoritară oferită de predicțiile indepente ale tuturor arborilor.

## 5. Workspace Setup

### Câteva biblioteci de care vom avea nevoie

In [ ]:
from math import log2
import csv
import os
import random

### Hiperparametrii necesari rulării

In [ ]:
DATASET_NAME = 'Tennis'  #@param ['Chess', 'Car', 'Tennis']

# Adâncimea arborilor
D = 7 #@param {type: "slider", min: 2, max: 10}

# Procentul de exemple din setul de date utilizat la construcția arborilor
P = 50 #@param {type: "slider", min: 1, max: 100}

### Funcții ajutătoare pentru descărcarea și lucrul cu setul de date

In [ ]:
class Node:
    """ Representation for a node from the decision tree """
    def __init__(self, label):
        """
            for non-leafs it is the name of the attribute
            for leafs it is the class
        """
        self.label = label
        
        # Dictionary of (attribute value, nodes)
        self.children = {}
    
    def display(self, string):
        print(string + self.label)
        string += "\t"
        if self.children:
            for key, value in self.children.items():
                print(string + key)
                value.display(string + "\t")


def getArchive(dataSetName):
    """ Checks if a specific dataset is present in the local directory, if not,
    downloads it.

    Args:
        dataSetName (str): the dataset name
    """
    dataset_file = "datasets" + os.sep + dataSetName.lower()
    print(dataset_file)
    
    from os import path
    if not path.isfile(dataset_file):
        import urllib
        print("Downloading...")
        urllib.request.urlretrieve(dataset_url, filename=dataset_file)
        assert(path.isfile(dataset_file))
        print("Got the archive")
    else:
        print(f"{dataset_file} already in the local directory")


def getDataSet(dataSetName):
    """ Reads a dataset

    Args:
        dataSetName (str): Name for the dataset

    Returns:
        A tuple containing (classes, attributes, examples):
        classes (set): the classes that are found in the dataset
        attributes (list of strings): the attributes for the dataset
        examples (list of dictionaries): one example contains an entry as
            (attribute name, attribute value)
    """

    dataset_file = "datasets" + os.sep + dataSetName.lower()

    f_in = open(dataset_file, 'r')
    csv_reader = csv.reader(f_in, delimiter=",")

    # Read the header row
    row = next(csv_reader)

    # The last element represents the class
    attributeNames = row[:-1]
    
    examples = []
    classes = set()

    for row in csv_reader:
        *attributes, label = row
        classes.add(label)
        example = dict(zip(attributeNames, attributes))
        example["CLASS"] = label
        examples.append(example)
    
    f_in.close()
    return classes, attributeNames, examples

### Descărcare și încarcare set de date

In [ ]:
getArchive(DATASET_NAME)
classes, attributes, examples = getDataSet(DATASET_NAME)
examples
attributes

datasets/tennis
datasets/tennis already in the local directory


['Outlook', 'Temperature', 'Humidity', 'Windy']

## 6. Cerințe

1. [3 pct] Implementați o funcție recursivă *randomTree* care construiește arbori de decizie de adâncime **d** pe baza unui **set de date X** și a unei **mulțimi de atribute A** astfel:
 * Dac̆a *d = 0*, atunci se construiește un nod frunză cu clasa majoritară din X.
 * Dacă *d > 0*, atunci se alege aleator un atribut $a_i$ din A și se construiește câte un subarbore pentru fiecare valoare $v_j$ a atributului $a_i$ apelând *randomTree* pentru *d − 1*:
$$
X_{i/j} = \{x \in X|a_{i}(x) = v_k\}\\
A_{new} = A \setminus \{a_i\}
$$

In [ ]:
def randomTree(d, X, A):
   # Cerință 1
   if d == 0:
     return Node(majClass(X))
   randomA = random.choice(A)
   auxA = A[:]
   auxA.remove(randomA)
   uniqueVals = []
   auxX = X
   nod = Node(randomA)
   for elem in X:
     if not elem[randomA] in uniqueVals:
      nod.children[elem[randomA]] = randomTree(d-1, filterX(X, randomA, elem[randomA]), auxA)
      uniqueVals.append(elem[randomA])
   return nod

def filterX(X, key, val):
  result = []
  for elem in X:
    if elem[key] == val:
      result.append(elem)
  return result

def majClass(X):
  classCount = {}
  for elem in classes:
    classCount[elem] = 0
  for elem in X:
    clasa = elem["CLASS"]
    classCount[clasa] = classCount[clasa] + 1
  maxCount = -1
  maxClass = ""
  for elem in classCount:
    if maxCount < classCount[elem]:
       maxCount = classCount[elem]
       maxClass = elem
  return maxClass

#testare
auxAtt = attributes[:]
auxEx = examples[:]
root = randomTree(3, auxEx, auxAtt)
root.display("- ")


- Windy
- 	false
- 		Temperature
- 			hot
- 				Outlook
- 					sunny
- 						no
- 					overcast
- 						yes
- 			mild
- 				Humidity
- 					high
- 						no
- 					normal
- 						yes
- 			cool
- 				Humidity
- 					normal
- 						yes
- 	true
- 		Humidity
- 			high
- 				Temperature
- 					hot
- 						no
- 					mild
- 						no
- 			normal
- 				Temperature
- 					cool
- 						no
- 					mild
- 						yes


2. [3 pct] Implementați o funcție recursivă *id3* care construiește arbori de decizie pe baza unui **set de date X** și a unei **mulțimi de atribute A**.
    
  Trebuie avute în vedere următoarele aspecte:
  * dacă toate exemplele din X aparțin unei singure clase C, atunci se construiește un nod frunză etichetat cu acea clasă C
  * dacă nu mai exista atribute, atunci construiește nodul frunză etichetat cu cea mai frecventă clasă din X
    
  În caz contrar:
  * se alege atributul $a^*$ care aduce cel mai mai mare câștig informațional și se construiește un *nod intermediar* corespunzător acestuia.

  $$
    entropy(X) = -\sum_{c \in C}\frac{|X_c|}{|X|}log_2\frac{|X_c|}{|X|}
  $$
  $$
    gain(X, a) = entropy(X) - \sum_{v_{j} \in vals(a)} \frac{|X_{i/j}|}{|X|}entropy(X_{i/j})
  $$
  $$
    a^* = \underset{a \in A}{\operatorname{arg max}}\ gain(X, a)
  $$

  * pentru fiecare valoare posibilă $v_j$ a lui $a^*$ se construiește un subarbore apelând recursiv funcția *id3* pentru:

$$
  X_j = \{x \in X|a^*(x) = v_j\}\\
  A_{new} = A\setminus\{a^*\}
$$

În cazul prezentat mai sus, entropia este utilizată pentru a măsura randomness-ul din date. Intuitiv, cu cât un eveniment are probabilitate mai mare să se întâmple atunci acesta va avea o entropia din ce în ce mai mică. Prin modul în care se construiește arborele *ID3* se încearcă reducerea entropiei alegând la fiecare pas atributele care ne ofera cea mai multă informație. Cât considerați că este entropia într-un *nod frunză*?

In [ ]:
def mostFrequentClass(X):
    # TODO Cerință 2
    return majClass(X)


def entropy(X):
    # TODO Cerință 2
    entropy = 0
    totalClasses = len(X)
    classCount = {}
    for elem in classes:
      classCount[elem] = 0
    for elem in X:
      clasa = elem["CLASS"]
      classCount[clasa] = classCount[clasa] + 1
    
    for elem in classes:
      if classCount[elem] != 0:
        entropy = entropy - ((classCount[elem]/totalClasses) * log2(classCount[elem]/totalClasses))
    return entropy

def gain(X, A):
    # TODO Cerință 2
    gain = entropy(X)
    uniqueSplits = []
    for elem in X:
      if not elem[A] in uniqueSplits:
        uniqueSplits.append(elem[A])
    for elem in uniqueSplits:
      Xij = filterX(X, A, elem)
      gain = gain - (len(Xij) / len(X)) * entropy(Xij)
    return gain

def isLeaf(X, A):
  if len(A) == 0:
    return True
  firstClass = X[0]["CLASS"]
  for elem in X:
    if elem["CLASS"] != firstClass:
      return False
  return True

def id3(X, A):
    # TODO Cerință 2
    if isLeaf(X, A) == True:
      return Node(majClass(X))
    bestAttr = A[0]
    bestGain = -1

    for elem in A:
      newGain = gain(X, elem)
      if bestGain < newGain:
        bestGain = newGain
        bestAttr = elem

    auxA = A[:]
    auxA.remove(bestAttr)
    uniqueVals = []
    auxX = X
    tree = Node(bestAttr)
    for elem in X:
      if not elem[bestAttr] in uniqueVals:
        tree.children[elem[bestAttr]] = id3(filterX(X, bestAttr, elem[bestAttr]), auxA)
        uniqueVals.append(elem[bestAttr])  
    
    return tree

treeId3 = id3(auxEx, auxAtt)
treeId3.display("")

def evaluate(tree, example):
    # TODO Cerință 2
    # Functia intoarce clasa prezisa de arborele `tree` pentru exemplul `example`
    if type(tree) == list:
      return evaluateForest(tree, example)
    else:
      return evaluateTree(tree, example)

def evaluateTree(tree, example):
  if not bool(tree.children) == True:
    return tree.label
  currLabel = tree.label
  currBranch = example[currLabel]
  if currBranch in tree.children:
    return evaluate(tree.children[currBranch], example)
  return 'NoData'

print("\nPredictia pentru prima valoare este " + evaluate(root2, auxEx[0]))

def precision(tree, X, c):
    prec = 0
    predicted_ct = 0
    
    for ex in X:
        pred_c = evaluate(tree, ex)
        if pred_c == c:
            predicted_ct += 1
            if ex['CLASS'] ==c:
                prec += 1
    
    if predicted_ct != 0:
        return prec / predicted_ct
    return 0
    

def recall(tree, X, c):
    X_c = list(filter(lambda ex: ex['CLASS'] == c, X))
    recall = 0
    
    for ex in X_c:
        pred_c = evaluate(tree, ex)
        if pred_c == c:
            recall += 1
            
    recall /= len(X_c)
    return recall
    
def accuracy(tree, X):
  count = 0
  for x in X:
    if evaluate(tree, x) == x['CLASS']:
      count += 1
  return 1.0 * count / len(X)
    

Outlook
	sunny
		Humidity
			high
				no
			normal
				yes
	overcast
		yes
	rain
		Windy
			false
				yes
			true
				no

Predictia pentru prima valoare este no


3. [4 pct] Implementați clasificator de tip pădure de arbori aleatori construind *n* arbori de adâncime maximă *d* fiecare dintre aceștia pornind de la o submulțime aleatoare a lui X.

    Folosiți funcția *randomTree* de la cerința 1.
  * Porniți de la *n = 100*, *d = 3* și submulțimi formate din 50% din elementele lui X alese la întamplare și experimentați cu acești hiperparametrii.
  * Pentru predicția clasei pentru obiecte noi alegeți clasa majoritară
  * Comparați rezultatele obținute folosind un singur arbore construit cu ID3 și o pădure de arbori aleatori. Discuție după *zgomot*, *overfitting*.

In [ ]:
def randomForest(X, n, d):
    # TODO Cerință 3
    trees = []
    for i in range(n):
      Attr = attributes[:]
      newSize = len(X) * P / 100
      newX = random.sample(X, int(newSize))
      trees.append(randomTree(d, newX, Attr))
    return trees
    
def evaluateForest(trees, example):
  results = []
  for tree in trees:
    foundClass = evaluate(tree, example)
    if foundClass != "NoData":
      results.append(foundClass)
  bestClass = ''
  bestCount = 0
  uniques = {}
  for elem in results:
    if elem not in uniques: 
      uniques[elem] = results.count(elem)
      if uniques[elem] > bestCount:
        bestCount = uniques[elem]
        bestClass = elem
  return bestClass


In [ ]:
# testare
# treeId3 este arborele generat folosind id3
randForest = randomForest(auxEx, 10, 3)
print("Id3 ------- Random Forest")
print("\nAccuracy")
print(str(accuracy(treeId3, auxEx)) + " ------- " + str(accuracy(randForest, auxEx)))
for clasa in classes:
  print("\nClasa " + clasa)
  print("Precision")
  print(str(precision(treeId3, auxEx, clasa)) + " ------- " + str(precision(randForest, auxEx, clasa)))
  print("Recall")
  print(str(recall(treeId3, auxEx, clasa)) + " ------- " + str(recall(randForest, auxEx, clasa)))
  




Id3 ------- Random Forest

Accuracy
1.0 ------- 0.9285714285714286

Clasa no
Precision
1.0 ------- 0.8333333333333334
Recall
1.0 ------- 1.0

Clasa yes
Precision
1.0 ------- 1.0
Recall
1.0 ------- 0.8888888888888888


## 7. Set de date

În cadrul acestui laborator se vor folosi seturile de date [Car Evaluation](https://archive.ics.uci.edu/ml/datasets/Car+Evaluation), [Chess](https://archive.ics.uci.edu/ml/datasets/Chess+%28King-Rook+vs.+King-Pawn%29) și [Tennis](https://www.kaggle.com/fredericobreno/play-tennis).

Aceste seturi de date sunt "ușor" modificate astfel încât pe prima linie să se afle atributele și labelul/clasa din care face parte fiecare exemplu.

Atributele datasetului *Chess* nu sunt intuitive, iar dacă doriți să aflați mai multe informații despre acestea, puteți accesa link-ul de [aici](https://pdfs.semanticscholar.org/db58/88d3f373aff2c6bd7b2f956b81c6896874a9.pdf?_ga=2.193733611.798337455.1582711694-486327444.1582711694).

## 8. Extra

### 8.1 ID3 exemplu
Un exemplu mai detaliat pentru construcția arborelui de decizie ID3 se poate găsi [aici](https://github.com/cs-pub-ro/ML/blob/master/lab/lab2/id3_example.pdf).

### 8.2 CART
Un alt algoritm utilizat poartă denumirea de CART (eng. Classification and Regression Tree). Dacă **ID3** utilizeaza **câștigul informațional (eng. information gain)**, **CART** utilizeaza o altă metrică numită **index-ul Gini (eng. Gini index sau Gini impurity)**.

Pentru implementare, se urmăresc exact aceeași [pași ca la ID3](#scrollTo=rjYqUPSbe1gG), singura diferentă fiind modul în care se calculează atributul utilizat într-un *nod intermediar*.
$$
Gini(X, a) = 1 - \sum_{c \in C}{p(c | attr(X) = a) ^2}
\\
a^* = \underset{a \in A}{\operatorname{arg min}}\ Gini(X, a)
$$